In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

from spectools import lpsd

from multiprocessing import Pool
pool = Pool()

In [ ]:
def fftnoise(f):
    f = np.array(f, dtype='complex')
    Np = (len(f) - 1) // 2
    phases = np.random.rand(Np) * 2 * np.pi
    phases = np.cos(phases) + 1j * np.sin(phases)
    f[1:Np+1] *= phases
    f[-1:-1-Np:-1] = np.conj(f[1:Np+1])
    return np.fft.ifft(f).real
    
def band_limited_noise(min_freq, max_freq, samples=1024, samplerate=1):
    freqs = np.abs(np.fft.fftfreq(samples, 1/samplerate))
    f = np.zeros(samples)
    idx = np.where(np.logical_and(freqs>=min_freq, freqs<=max_freq))[0]
    f[idx] = 1
    return fftnoise(f)
    
N  = int(1e6)
fs = 2000

x1 = 200*band_limited_noise(200, 500, N, fs)
x2 = 20*band_limited_noise(100, 200, N, fs)

fig, ax = plt.subplots(figsize=(6,4), dpi=150)
ax.set_xlabel('Sample')
ax.set_ylabel('Signal')
ax.plot(x1)
ax.plot(x2)

f1, psd01 = signal.welch(x1, fs=fs, nperseg=200, noverlap=100, scaling='spectrum', return_onesided=True)
f2, psd02 = signal.welch(x2, fs=fs, nperseg=200, noverlap=100, scaling='spectrum', return_onesided=True)

fig, ax = plt.subplots(figsize=(6,4), dpi=150)
ax.set_xlabel('Fourier frequency (Hz)')
ax.set_ylabel(r'Power spectrum (${\rm units}^2/{\rm Hz}$)')
ax.loglog(f1, psd01)
ax.loglog(f2, psd02, linestyle='--')

In [ ]:
y = (x1,x2)

In [ ]:
bmin = 0
order = -1
lpsd.use_c_core = False

psd1 = lpsd.lpsd(y, fs=fs, bmin=bmin, order=order, pool=pool, object_return=True)

In [ ]:
bmin = 0
order = -1
lpsd.use_c_core = True

psd2 = lpsd.lpsd(y, fs=fs, bmin=bmin, order=order, object_return=True)

In [ ]:
fig, ax = plt.subplots(figsize=(6,4), dpi=150)
ax.set_xlabel('Fourier frequency (Hz)')
ax.set_ylabel(r'Power spectrums ${\rm units}^2/{\rm Hz}$')
ax.loglog(f1, psd01)
ax.loglog(f2, psd02)
ax.loglog(psd1.f, psd1.XX)
ax.loglog(psd2.f, psd2.XX, linestyle='--')
ax.loglog(psd1.f, psd1.YY)
ax.loglog(psd2.f, psd2.YY, linestyle='--')

In [ ]:
fig, ax = plt.subplots(figsize=(6,4), dpi=150)
ax.set_xlabel('Fourier frequency (Hz)')
ax.set_ylabel(r'Cross spectral density (units $/\sqrt{\rm Hz}$)')
ax.loglog(psd1.f, np.sqrt(np.abs(psd1.Gxy)))
ax.loglog(psd2.f, np.sqrt(np.abs(psd2.Gxy)), linestyle='--')